In [8]:
import pandas as pd
import numpy as np
fp = "aadhaar_master_merged.xls"
df = pd.read_csv(fp)
df.head()
df.shape


(2951501, 16)

In [10]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)
df['date'] = pd.to_datetime(df['date'])

dfd = df.drop_duplicates()
dfd.shape


(2332531, 16)

In [12]:
df['total_enrollments'] = (
    df['age_0_5'] +
    df['age_5_17'] +
    df['age_18_greater']
)
df['total_demo_updates'] = (
    df['demo_age_5_17'] +
    df['demo_age_17_']
)
df['total_bio_updates'] = (
    df['bio_age_5_17'] +
    df['bio_age_17_']
)


In [13]:
df.head

<bound method NDFrame.head of               date                        state        district  pincode  \
0       2025-03-01    Andaman & Nicobar Islands        Andamans   744101   
1       2025-03-01  Andaman And Nicobar Islands         Nicobar   744301   
2       2025-03-01  Andaman And Nicobar Islands         Nicobar   744301   
3       2025-03-01  Andaman And Nicobar Islands         Nicobar   744302   
4       2025-03-01  Andaman And Nicobar Islands         Nicobar   744303   
...            ...                          ...             ...      ...   
2951496 2025-12-31                  West Bengal  West Midnapore   721426   
2951497 2025-12-31                  West Bengal  West Midnapore   721504   
2951498 2025-12-31                  West Bengal  West Midnapore   721506   
2951499 2025-12-31                  West Bengal  West Midnapore   721507   
2951500 2025-12-31                  West Bengal  West Midnapore   721517   

         age_0_5  age_5_17  age_18_greater  demo_age_5_17

In [17]:

df = df.drop('has_enrol', axis=1)

In [18]:
print(df.columns.tolist())

['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17', 'bio_age_17_', 'has_demo', 'has_bio', 'has_enroll', 'total_updates', 'total_enrollments', 'total_demo_updates', 'total_bio_updates']


In [20]:
df['atpi'] = (
    df['demo_age_5_17'] +
    df['demo_age_17_'] +
    df['total_bio_updates']
) / (df['total_enrollments'] + 1)


In [21]:
df['dvi'] = (
    df['total_demo_updates']
) / (df['total_enrollments'] + 1)


In [22]:
df['bsi'] = (
    df['total_bio_updates']
) / (df['total_demo_updates'] + 1)


In [24]:
df = df.sort_values(['pincode', 'date'])
df.head

<bound method NDFrame.head of               date   state        district  pincode  age_0_5  age_5_17  \
149020  2025-09-02  100000          100000   100000      0.0       0.0   
181306  2025-09-03  100000          100000   100000      0.0       0.0   
310400  2025-09-08  100000          100000   100000      0.0       0.0   
342703  2025-09-09  100000          100000   100000      0.0       0.0   
406850  2025-09-11  100000          100000   100000      0.0       0.0   
...            ...     ...             ...      ...      ...       ...   
2807354 2025-12-26   Bihar  East Champaran   855456      0.0       0.0   
2807355 2025-12-26   Bihar  East Champaran   855456      0.0       0.0   
2879242 2025-12-28   Bihar  East Champaran   855456      0.0       0.0   
2910161 2025-12-29   Bihar  East Champaran   855456      0.0       0.0   
2910162 2025-12-29   Bihar  East Champaran   855456      0.0       0.0   

         age_18_greater  demo_age_5_17  demo_age_17_  bio_age_5_17  ...  \
149020

In [25]:
df['atpi_14d'] = (
    df.groupby('pincode')['atpi']
      .rolling(14, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)

df['dvi_14d'] = (
    df.groupby('pincode')['dvi']
      .rolling(14, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)

df['bsi_14d'] = (
    df.groupby('pincode')['bsi']
      .rolling(14, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)


In [26]:
def norm(col):
    return (col - col.min()) / (col.max() - col.min() + 1e-6)

df['atpi_n'] = norm(df['atpi_14d'])
df['dvi_n']  = norm(df['dvi_14d'])
df['bsi_n']  = norm(df['bsi_14d'])


In [27]:
df['aadhaar_load_score'] = (
    0.4 * df['atpi_n'] +
    0.3 * df['dvi_n'] +
    0.3 * df['bsi_n']
)


In [29]:
import numpy as np

conds = [
    df['aadhaar_load_score'] < 0.4,
    (df['aadhaar_load_score'] >= 0.4) & (df['aadhaar_load_score'] < 0.7),
    df['aadhaar_load_score'] >= 0.7
]

labels = ['Normal', 'Upcoming Surge', 'Critical Overload']

# Adding a default value (empty string) that matches the string data type of labels
df['load_status'] = np.select(conds, labels, default='Unknown')

In [34]:
phase1_cols = [
    'date', 'state', 'district', 'pincode',
    'total_enrollments', 'total_demo_updates', 'total_bio_updates',
    'atpi', 'dvi', 'bsi',
    'aadhaar_load_score', 'load_status'
]

phase1_df = df[phase1_cols]
phase1_df.head(50)


,date,state,district,pincode,total_enrollments,total_demo_updates,total_bio_updates,atpi,dvi,bsi,aadhaar_load_score,load_status
149020,2025-09-02,100000,100000,100000,3.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
181306,2025-09-03,100000,100000,100000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
310400,2025-09-08,100000,100000,100000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
342703,2025-09-09,100000,100000,100000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
406850,2025-09-11,100000,100000,100000,2.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
435134,2025-09-12,100000,100000,100000,2.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
629268,2025-09-19,100000,100000,100000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
656626,2025-09-20,100000,100000,100000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
1020317,2025-10-24,100000,100000,100000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal
1759533,2025-11-15,100000,100000,100000,3.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Normal


In [35]:
phase1_df.to_csv("phase1_aadhaar_pressure_metrics.csv", index=False)


In [36]:
df['aadhaar_load_score'].describe()


count    2.951501e+06
mean     2.373028e-03
std      9.207692e-03
min      0.000000e+00
25%      2.105754e-04
50%      3.627323e-04
75%      7.699449e-04
max      7.000283e-01
Name: aadhaar_load_score, dtype: float64

In [37]:
p40 = df['aadhaar_load_score'].quantile(0.40)
p75 = df['aadhaar_load_score'].quantile(0.75)
p90 = df['aadhaar_load_score'].quantile(0.90)


In [39]:
df['load_status'] = np.select(
    [
        df['aadhaar_load_score'] <= p40,
        (df['aadhaar_load_score'] > p40) & (df['aadhaar_load_score'] <= p75),
        df['aadhaar_load_score'] > p75
    ],
    [
        'Normal',
        'Emerging Stress',
        'Critical Overload'
    ],
    default='Unknown'
)


In [40]:
df['load_status'].value_counts(normalize=True)


load_status
Normal               0.40
Emerging Stress      0.35
Critical Overload    0.25
Name: proportion, dtype: float64

In [43]:
phase1_df.head(70)


,date,state,district,pincode,total_enrollments,total_demo_updates,total_bio_updates,atpi,dvi,bsi,aadhaar_load_score,load_status
149020,2025-09-02,100000,100000,100000,3.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,Normal
181306,2025-09-03,100000,100000,100000,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,Normal
310400,2025-09-08,100000,100000,100000,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,Normal
342703,2025-09-09,100000,100000,100000,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,Normal
406850,2025-09-11,100000,100000,100000,2.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...
546348,2025-09-16,Delhi,New Delhi,110001,2.0,9.0,10.0,6.333333,3.0,1.000000,0.000237,Normal
573264,2025-09-17,Delhi,Central Delhi,110001,1.0,5.0,1.0,3.000000,2.5,0.166667,0.000238,Normal
573283,2025-09-17,Delhi,New Delhi,110001,1.0,6.0,17.0,11.500000,3.0,2.428571,0.000249,Normal
604661,2025-09-18,Delhi,Central Delhi,110001,1.0,1.0,0.0,0.500000,0.5,0.000000,0.000237,Normal


In [46]:
# -------------------------------
# STEP 1: Log-scale pressure indices (recommended)
# -------------------------------
df['atpi_log'] = np.log1p(df['atpi'])
df['dvi_log']  = np.log1p(df['dvi'])
df['bsi_log']  = np.log1p(df['bsi'])

# -------------------------------
# STEP 2: Normalize
# -------------------------------
def norm(col):
    return (col - col.min()) / (col.max() - col.min() + 1e-6)

df['atpi_n'] = norm(df['atpi_log'])
df['dvi_n']  = norm(df['dvi_log'])
df['bsi_n']  = norm(df['bsi_log'])

# -------------------------------
# STEP 3: Composite Aadhaar Load Score
# -------------------------------
df['aadhaar_load_score'] = (
    0.4 * df['atpi_n'] +
    0.3 * df['dvi_n'] +
    0.3 * df['bsi_n']
)

# -------------------------------
# STEP 4: Percentile-based Load Classification
# -------------------------------
p40 = df['aadhaar_load_score'].quantile(0.40)
p75 = df['aadhaar_load_score'].quantile(0.75)

df['load_status'] = np.select(
    [
        df['aadhaar_load_score'] <= p40,
        (df['aadhaar_load_score'] > p40) & (df['aadhaar_load_score'] <= p75),
        df['aadhaar_load_score'] > p75
    ],
    [
        'Normal',
        'Emerging Stress',
        'Critical Overload'
    ],
    default='Unknown'  # Added a default value with the same string data type
)

# -------------------------------
# STEP 5: Final Phase-1 Output Columns
# -------------------------------
phase1_df = df[
    [
        'date', 'state', 'district', 'pincode',
        'total_enrollments',
        'total_demo_updates',
        'total_bio_updates',
        'atpi', 'dvi', 'bsi',
        'aadhaar_load_score',
        'load_status'
    ]
]

# -------------------------------
# STEP 6: Export Final CSV
# -------------------------------
phase1_df.to_csv(
    "phase1_aadhaar_pressure_metrics_final.csv",
    index=False
)

print("✅ Phase 1 CSV exported successfully")

✅ Phase 1 CSV exported successfully


In [51]:
df['atpi_log'] = np.log1p(df['atpi'])
df['dvi_log']  = np.log1p(df['dvi'])
df['bsi_log']  = np.log1p(df['bsi'])
# -------------------------------
# STEP 2: Normalize stress signals
# -------------------------------
df['atpi_n'] = norm(df['atpi_log'])   # age-transition pressure
df['dvi_n']  = norm(df['dvi_log'])    # demographic volatility
df['bsi_n']  = norm(df['bsi_log'])    # biometric stress

# -------------------------------
# STEP 3: Population pressure (latent risk)
# -------------------------------
df['population_pressure'] = np.log1p(df['total_enrollments'])
df['pop_n'] = norm(df['population_pressure'])

# -------------------------------
# STEP 4: FINAL Aadhaar Load Score
# -------------------------------
df['aadhaar_load_score'] = (
    0.30 * df['atpi_n'] +   # age-driven pressure
    0.25 * df['dvi_n'] +    # demographic churn
    0.25 * df['bsi_n'] +    # biometric stress
    0.20 * df['pop_n']      # latent population risk
)

# -------------------------------
# STEP 5: Percentile-based load classification
# -------------------------------
p40 = df['aadhaar_load_score'].quantile(0.40)
p75 = df['aadhaar_load_score'].quantile(0.75)

df['load_status'] = np.select(
    [
        df['aadhaar_load_score'] <= p40,
        (df['aadhaar_load_score'] > p40) & (df['aadhaar_load_score'] <= p75),
        df['aadhaar_load_score'] > p75
    ],
    [
        'Normal',
        'Emerging Stress',
        'Critical Overload'
    ],
    default='Unknown'  # Added a default value with string type to match the choices
)

# -------------------------------
# STEP 6: Final Phase-1 output table
# -------------------------------
phase1_final_df = df[
    [
        'date', 'state', 'district', 'pincode',
        'total_enrollments',
        'total_demo_updates',
        'total_bio_updates',
        'atpi', 'dvi', 'bsi',
        'population_pressure',
        'aadhaar_load_score',
        'load_status'
    ]
]

# -------------------------------
# STEP 7: Export CSV
# -------------------------------
phase1_final_df.to_csv(
    "phase1_aadhaar_pressure_metrics_FINAL.csv",
    index=False
)

print("✅ Phase 1 FINAL CSV exported successfully")

✅ Phase 1 FINAL CSV exported successfully


In [52]:
df['load_status'].value_counts(normalize=True)


load_status
Normal               0.401006
Emerging Stress      0.349122
Critical Overload    0.249872
Name: proportion, dtype: float64